# 基于百度人脸识别的ASCII码大头贴技术

在日常表情包互怼中，我们经常可以看到一些图片是由更小元素构成的，于是可将其运用到python中，便写了一个图片转ASCII码的脚本。原理其实很简单，通过对图片每个元素的r，g，b三个通道值进行加权求和可以得到灰度值，而不难得到ASCII码的灰度值，这样做一个转化即可。

本次采用某某作为例子，（附一）。利用百度的人脸检测可以得到人脸在图像中的具体位置，我们期待实现这个样子（附二），但现实总是残酷的，只运用百度人脸识别只能识别到脸，头的一部分被抛弃了，所以我们需要手动调整。好在百度的人脸识别给出了人脸各个位置的置信点（附三），所以根据这些点的位置就可大致获得整个头部的位置。

如果想直接将图片转化为ASCII码而不用到百度人脸识别技术，可跳到代码末'处理'部分中，调整'target'变量的几个参数即可。

**获得token**

In [1]:
import urllib, urllib2, sys
import ssl

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=ybS3fzfGnoGRXZNq9OCHTGZK&client_secret=G4U7D7qtG1cxENi78dKMQCtSG50AGGL6'
request = urllib2.Request(host)
request.add_header('Content-Type', 'application/json; charset=UTF-8')
response = urllib2.urlopen(request)
content = response.read()
if (content):
    print(content)

{"access_token":"24.8d18ac8fdfc665e88263028d33d41d08.2592000.1522230361.282335-10852496","session_key":"9mzdA86eeOYDJi7Up10wFafDkdgGsdcCgAVhxQ4Z2rZwcFbB0ABZaiRxxgXvkrX+QHU09FI6C3b72xTnqGhp7OgKhnzWEQ==","scope":"public vis-faceverify_faceverify vis-faceattribute_faceattribute vis-faceverify_faceverify_v2 vis-faceverify_faceverify_match_v2 brain_all_scope vis-faceverify_vis-faceverify-detect wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower bnstest_fasf lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission","refresh_token":"25.2f826329a0376f2ea62846072466b63d.315360000.1834998361.282335-10852496","session_secret":"55114aa2ef2d8c6e276707629f93527b","expires_in":2592000}



In [2]:
from aip import AipFace
import cv2
import base64
import json

# 百度云的KEY
APP_ID = '10852496'
API_KEY = 'ybS3fzfGnoGRXZNq9OCHTGZK'
SECRET_KEY = 'G4U7D7qtG1cxENi78dKMQCtSG50AGGL6'
client = AipFace(APP_ID, API_KEY, SECRET_KEY)

request_url = "https://aip.baidubce.com/rest/2.0/face/v1/detect"
ascii_char = list("$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. ")

#<--------------------------主代码部分-------------------------->
imgsrc = "example.jpg"          # 文件路径
max_width = 180            # 宽度最大字符数

# 图像到字符串的转化
def get_char(r,g,b):
    lenght = len(ascii_char)
    gray = int(0.2126 * r + 0.7152 * g + 0.0722 * b)
    # gray = int(0.3 * r + 0.59 * g + 0.11 * b)
    unit = (256.0 + 1)/lenght
    return ascii_char[int(gray/unit)]

# 二进制方式打开图片文件
f = open(imgsrc, 'rb')
img = base64.b64encode(f.read())

params = {"face_fields":"age,beauty,expression,faceshape,gender,glasses,landmark,race,qualities","image":img,"max_face_num":5}
params = urllib.urlencode(params)

# 请求人脸识别
access_token = '24.8d18ac8fdfc665e88263028d33d41d08.2592000.1522230361.282335-10852496'
request_url = request_url + "?access_token=" + access_token
request = urllib2.Request(url=request_url, data=params)
request.add_header('Content-Type', 'application/x-www-form-urlencoded')
response = urllib2.urlopen(request)
content = response.read()

# result为人脸识别结果的字典
if content:
    result = json.loads(content)

# 72个面部特征点的置信位置
landmark72 = result['result'][0]['landmark72']

# 提取我们需要的点
nose = landmark72[56]     # 鼻子
chin = landmark72[5]      # 下巴
l_brow = landmark72[22]   # 左眉角
r_brow = landmark72[43]   # 右眉角
l_brow2 = landmark72[23]  # 左眉
r_brow2 = landmark72[40]  # 右眉

# 重要变量含义：
# img:处理的原始图片
# l_border:处理后图片的边界在原图片中的位置，其余三边同理
# img(src)_new:经过裁剪后的图片(路径)
# target,width,height:经过压缩后的最终图片及其宽、高
# B,G,R:最终图片的三个通道矩阵

# cv2处理图片(尺寸)
img = cv2.imread(imgsrc)
l_border = 0 if 2*l_brow['x']<nose['x'] else 2*l_brow['x']-nose['x']
r_border = img.shape[1] if 2*r_brow['x']-nose['x']>img.shape[1] else 2*r_brow['x']-nose['x']
u_border = max(min(2*l_brow2['y']-nose['y'],2*r_brow2['y']-nose['y']),0)
d_border = chin['y']
# print l_border,r_border,u_border,d_border

# 裁剪
img_new = img[u_border:d_border,l_border:r_border]
src_new = "temp/new_"+imgsrc
cv2.imwrite(src_new, img_new)

# 处理
pre_width = r_border-l_border
pre_height = d_border-u_border
exact = pre_width*1.0/pre_height # 宽长比
width = max_width
height = int(round(width/(2*exact)))
target=cv2.resize(img_new,(width,height),interpolation=cv2.INTER_CUBIC)
cv2.imwrite('temp/target.jpg',target)
txt = ''
B = target[:, :, 0]
G = target[:, :, 1]
R = target[:, :, 2]
for x in range(0,height):
    for y in range(0,width):
        txt += get_char(R[x][y],G[x][y],B[x][y])
    txt += '\n'
with open("result.txt",'wb') as f:
    f.write(txt)

## 附录

图一：
<img src="img/example.jpg" style="height:500px">

图二：
<img src="img/example_target.png" style="height:500px">

图三：
<img src="img/ai_baidu.png" style="height:500px">